In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [1]:


# Dummy data
data_size = 1000
data_dim = 10
data = torch.randn(data_size, data_dim)

# Generator network
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, z):
        out = self.fc(z)
        out = self.relu(out)
        return out

# Discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        out = self.fc(x)
        return out

# Initialize the generator and discriminator
latent_dim = 20
generator = Generator(latent_dim, data_dim)
discriminator = Discriminator(data_dim)

# Set device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set up optimizers
optimizer_G = optim.RMSprop(generator.parameters(), lr=0.00005)
optimizer_D = optim.RMSprop(discriminator.parameters(), lr=0.00005)

# Set up loss function
criterion = nn.MSELoss()

# Training loop
num_epochs = 1000
batch_size = 64
for epoch in range(num_epochs):
    for i in range(0, data_size, batch_size):
        # Train the discriminator
        discriminator.zero_grad()

        # Real data
        real_samples = Variable(data[i:i + batch_size]).to(device)
        real_labels = Variable(torch.ones(batch_size, 1)).to(device)

        # Generate fake samples
        z = Variable(torch.randn(batch_size, latent_dim)).to(device)
        fake_samples = generator(z)
        fake_labels = Variable(torch.zeros(batch_size, 1)).to(device)

        # Discriminator loss on real data
        real_output = discriminator(real_samples)
        d_loss_real = torch.mean(real_output)

        # Discriminator loss on fake data
        fake_output = discriminator(fake_samples.detach())
        d_loss_fake = torch.mean(fake_output)

        # Total discriminator loss
        d_loss = -(d_loss_real - d_loss_fake)
        d_loss.backward()
        optimizer_D.step()

        # Clip discriminator weights
        for p in discriminator.parameters():
            p.data.clamp_(-0.01, 0.01)

        # Train the generator
        generator.zero_grad()

        # Generate fake samples
        z = Variable(torch.randn(batch_size, latent_dim)).to(device)
        fake_samples = generator(z)

        # Generator loss
        g_loss = -torch.mean(discriminator(fake_samples))
        g_loss.backward()
        optimizer_G.step()

    # Print progress
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], "
              f"D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}")


Epoch [100/1000], D_loss: -0.0165, G_loss: 0.0009
Epoch [200/1000], D_loss: -0.0072, G_loss: -0.0079
Epoch [300/1000], D_loss: -0.0080, G_loss: -0.0102
Epoch [400/1000], D_loss: -0.0065, G_loss: -0.0100
Epoch [500/1000], D_loss: -0.0067, G_loss: -0.0100
Epoch [600/1000], D_loss: -0.0072, G_loss: -0.0100
Epoch [700/1000], D_loss: -0.0072, G_loss: -0.0100
Epoch [800/1000], D_loss: -0.0072, G_loss: -0.0100
Epoch [900/1000], D_loss: -0.0072, G_loss: -0.0100
Epoch [1000/1000], D_loss: -0.0072, G_loss: -0.0100
